# Проект 0: "Github. Самый быстрый старт"


## Цель проекта

Цель этого проекта - выдумать и имплементировать алгоритм принятия решений, следуя которому, *Игрок* в среднем сможет победить в простой разговорной *Игре* на числа (правила которой указаны ниже) **быстрее чем за 30 ходов**.

Среднее количество *Попыток* (за 1000 *Игр*) и будет служить критерием того насколько хорош алгоритм: чем меньше *Попыток* - тем лучше алгоритм 👍⭐️.


## TL;DR

In [1]:
import numpy as np


def game_core_v4(secret_number, show_path=False):
    """Вернуть статистику действий Игрока, играющему по алгоритму v4.
    
    Именованные аргументы:
    secret_number -- число которое загадал Вода
    show_path -- булева - показывать ли в результате путь из
                 ПредположенныхЧисел которые придумывал Игрок: эта
                 часть статистики очень полезна для дебага но не
                 особенно нужна в остальных случаях (default False)
    
    На выходе - словарь со статистикой Игрока за одну игру.
    Поля словаря:
        is_win -- булева - победил ли Игрок? False если Сдался.
        attempt_count -- число - сколько Попыток сделал Игрок
        path -- список чисел - (опциональное поле - оно не показывается
                если аргумент show_path равен False) - все
                ПредположенныеЧисла которые обдумывал Игрок; в некоторых
                случаях тут может быть больше чисел чем было сделано
                Попыток - потому что Игрок не все числа решился Огласить
                вслух.
    
    Алгоритм:
    =========
    Начинаем игру с числа 2**6.
    - МЕНЬШЕ тогда пробуем (2**6 - 2**5)
    - РАВНО успех - ПОБЕДА
    - БОЛЬШЕ тогда пробуем (2**6 + 2**5)
    
    Если мы ещё не достигли успеха - просто понижаем степень.
    
    Если наше ПредположенноеЧисло вылезает за пределы [1; 100], мы НЕ
    ОГЛАШАЕМ его вслух и смотрим:
    - Если Предположенное число меньше 1, то считаем что результат от
      Воды был БОЛЬШЕ.
    - Если ПредположенноеЧисло больше верхнего предела, считаем что
      результат от Воды был МЕНЬШЕ.
    В любом из этих двух случаев - продолжить двигаться по алгоритму,
    пока ПредположенноеЧисло не попадёт наконец в интервал [1; 100].
    
    На всякий случай чтобы не впасть в рекурсию - ставим себе
    ограничение если мы почему-то посчитали ПредположенноеЧисло 12 раз и
    до сих пор не угадали результат - то надо Сдаться.
    """
    
    limit_min = 1
    limit_max = 100
    attempt_count = 0
    path = [] # Тут будут храниться все ПредположенныеЧисла.
    
    
    def declare(guess_number):
        """Огласить догадку вслух, истратив Попытку Игрока."""
        nonlocal attempt_count
        
        attempt_count += 1
        
        if guess_number < secret_number:
            return 'less'
        elif guess_number == secret_number:
            return 'equal'
        else:
            return 'more'
    
    
    # С точки зрения цикла можно считать что это - начальное положение
    # алгоритма.
    is_win = False # Если True - Игрок Победил, если False - Проиграл.
    guess_number = 0
    declaration = 'less' # ПредположенноеЧисло ... чем СекретноеЧисло.
    power = 6
    despair_level = 0 # Если Отчаянье достигнет 12 - Игрок сдастся.
    while despair_level < 12:
        
        despair_level += 1
        
        if declaration == 'equal':
            is_win = True
            break
        if declaration == 'less':
            guess_number += 2**power
        if declaration == 'more':
            guess_number -= 2**power
        
        path.append(guess_number)
        
        power -= 1
        
        # Если мы выбились за пределы.
        if guess_number < limit_min:
            declaration = 'less'
            continue
        if guess_number > limit_max:
            declaration = 'more'
            continue
        
        declaration = declare(guess_number)
    
    result = {
        'is_win': is_win,
        'attempt_count': attempt_count,
    }
    
    if show_path:
        result['path'] = path # Опциональная статистика.
    
    return result


def score_game(game_core):
    """Вернуть результаты проверки алгоритма угадывания.
    
    Именованные аргументы:
    game_core -- функция которая сможет эмулировать решения Игрока
    
    Запускаем игру 1000 раз, чтобы узнать, как быстро игра угадывает
    число.
    """
    results = []
    
    # Фиксируем RANDOM SEED, чтобы эксперимент был воспроизводимым!
    np.random.seed(1)
    secret_numbers = np.random.randint(1,101, size=1000)
    
    for secret_number in secret_numbers:
        results.append(game_core(secret_number))
    
    win_results = [x for x in results if x['is_win']]
    loss_results = [x for x in results if not x['is_win']]
    
    win_ratio = round(
        len(win_results) / len(secret_numbers),
        ndigits=4
        )
    loss_ratio = round(
        len(loss_results) / len(secret_numbers),
        ndigits=4
        )
    
    win_scores = [x['attempt_count'] for x in win_results]
    mean_win_score = round(np.mean(win_scores), 2)
    
    print()
    print()
    print(f'Ваш алгоритм:')
    print(f'* угадывает число в среднем за {mean_win_score} попыток')
    print(f'* проигрывает в среднем в {loss_ratio*100}% случаев')
    
    return {
        'win_ratio': win_ratio,
        'loss_ratio': loss_ratio,
        'mean_win_score': mean_win_score,
        }


score_game(game_core_v4)



Ваш алгоритм:
* угадывает число в среднем за 5.99 попыток
* проигрывает в среднем в 0.0% случаев


{'win_ratio': 1.0, 'loss_ratio': 0.0, 'mean_win_score': 5.99}

`угадывает число в среднем за 5.99 попыток` - это быстрее чем `в среднем за 30 попыток` с которым я соревнуюсь! Похоже что я достиг успеха в своей цели ⭐️